# Animal Shelter EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Reading the CSVs

In [2]:
df_intakes = pd.read_csv('../raw_data/Austin_Animal_Center_Intakes.csv', parse_dates=['DateTime'])
df_outcomes = pd.read_csv('../raw_data/Austin_Animal_Center_Outcomes.csv', parse_dates=['DateTime'])
df_straymap = pd.read_csv('../raw_data/Austin_Animal_Center_Stray_Map.csv')

#### Table sizes

In [3]:
print(df_intakes.shape)
print(df_outcomes.shape)
print(df_straymap.shape)

(124120, 12)
(124491, 12)
(16, 10)


<b>Straymap only have 16 rows so we will not use this table.</b>

## Merging Intakes and Outcomes   

Merging the Intakes dataset with the Outcomes dataset.

In [4]:
df_intakes.drop_duplicates(inplace = True)
print(df_intakes.shape)
df_intakes.head(2)

(124101, 12)


,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,2019-01-03 16:19:00,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,2015-07-05 12:59:00,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver


In [5]:
df_outcomes.drop_duplicates(inplace = True)
print(df_outcomes.shape)
df_outcomes.head(2)

(124474, 12)


,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown


### Dropping and sorting before merging datasets   
Next steps we will:
- Dropping irrelevant features
- Sorting all the intakes and outcomes based on Animal ID and DateTime
- Dropping all the animals that were more than one time in the shelter

In [6]:
#Dropping irrelevant features

df_intakes.drop(columns = ['Name','MonthYear','Found Location','Sex upon Intake'], inplace = True)
df_intakes.rename(columns = {'DateTime':'DateTimeIntake'}, inplace = True)
df_outcomes.drop(columns = ['Name','MonthYear','Date of Birth','Breed','Color','Animal Type', 'Outcome Subtype'], inplace = True)
df_outcomes.rename(columns = {'DateTime':'DateTimeOutcome'}, inplace = True)

In [7]:
# Sorting all the intakes and outcomes based on Animal ID and DateTime

df_intakes.sort_values(by = ['Animal ID', 'DateTimeIntake'], ascending = [True, True], inplace = True)
df_outcomes.sort_values(by = ['Animal ID','DateTimeOutcome'], ascending = [True,True], inplace = True) 

In [8]:
# Dropping all the animals that were more than one time in the shelter

df_intakes.drop_duplicates(subset = 'Animal ID', inplace = True)
df_outcomes.drop_duplicates(subset = 'Animal ID', inplace = True)

# Merging the datasets
df_merged = pd.merge(left = df_intakes, right = df_outcomes, how = 'left', on = ['Animal ID'])
df_merged.head(2)

,Animal ID,DateTimeIntake,Intake Type,Intake Condition,Animal Type,Age upon Intake,Breed,Color,DateTimeOutcome,Outcome Type,Sex upon Outcome,Age upon Outcome
0,A006100,2014-03-07 14:26:00,Public Assist,Normal,Dog,6 years,Spinone Italiano Mix,Yellow/White,2014-03-08 17:10:00,Return to Owner,Neutered Male,6 years
1,A047759,2014-04-02 15:55:00,Owner Surrender,Normal,Dog,10 years,Dachshund,Tricolor,2014-04-07 15:12:00,Transfer,Neutered Male,10 years


### Creating our target (Days in Shelter)   
Next we will create our target with OutcomeDate - IntakeDate.

In [9]:
# Filtering only dogs

df_filtered = df_merged[(df_merged['Animal Type'] == 'Dog') | (df_merged['Animal Type'] == 'Cat')].copy()

# Calculating the number of days a dog stays in shelter

df_filtered['days_in_shelter'] = np.ceil((df_filtered['DateTimeOutcome'] - df_filtered['DateTimeIntake']) / np.timedelta64(24,'h'))

# Dropping all the the negatives values (errors in merging datasets) and null values (dogs that are still in shelter)

df_filtered = df_filtered[df_filtered.days_in_shelter > 0]
df_filtered.head(2)

,Animal ID,DateTimeIntake,Intake Type,Intake Condition,Animal Type,Age upon Intake,Breed,Color,DateTimeOutcome,Outcome Type,Sex upon Outcome,Age upon Outcome,days_in_shelter
0,A006100,2014-03-07 14:26:00,Public Assist,Normal,Dog,6 years,Spinone Italiano Mix,Yellow/White,2014-03-08 17:10:00,Return to Owner,Neutered Male,6 years,2.0
1,A047759,2014-04-02 15:55:00,Owner Surrender,Normal,Dog,10 years,Dachshund,Tricolor,2014-04-07 15:12:00,Transfer,Neutered Male,10 years,5.0


## Data Preprocessing   

In [10]:
# Type of each feature
#df_filtered.dtypes

#Dropping rows with NaNs
df_filtered.dropna(inplace = True)
print(df_filtered.shape)
df_filtered.info()

(102725, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 102725 entries, 0 to 110888
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         102725 non-null  object        
 1   DateTimeIntake    102725 non-null  datetime64[ns]
 2   Intake Type       102725 non-null  object        
 3   Intake Condition  102725 non-null  object        
 4   Animal Type       102725 non-null  object        
 5   Age upon Intake   102725 non-null  object        
 6   Breed             102725 non-null  object        
 7   Color             102725 non-null  object        
 8   DateTimeOutcome   102725 non-null  datetime64[ns]
 9   Outcome Type      102725 non-null  object        
 10  Sex upon Outcome  102725 non-null  object        
 11  Age upon Outcome  102725 non-null  object        
 12  days_in_shelter   102725 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(10)
m

In [11]:
# Transforming ages in string to integer in months

def dog_age(df):
    dog_months = []
    dog_years = []
    for age in df:
        if 'years' in age or 'year' in age:
            years = int(re.findall('(-?\d+)',age)[0])
            if years < 0:
                dog_months.append(np.nan)
                dog_years.append(np.nan)
            else:
                dog_months.append(years*12)
                dog_years.append(years)
        elif 'months' in age or 'month' in age:
            months = int(re.findall('(-?\d+)',age)[0])
            years = int(re.findall('(-?\d+)',age)[0])
            if months < 0:
                dog_months.append(np.nan)
            else:
                dog_months.append(months)
                dog_years.append(1)
        else:
            weeks_or_days = int(re.findall('(-?\d+)',age)[0])
            if weeks_or_days < 0:
                dog_months.append(np.nan)
            else:
                dog_months.append(1)
                dog_years.append(1)
    return [dog_months, dog_years]

In [12]:
#Converting dog ages
df_filtered['age_upon_intake_months'] = dog_age(df_filtered['Age upon Intake'])[0]
df_filtered['age_upon_intake_years'] = dog_age(df_filtered['Age upon Intake'])[1]
df_filtered['age_upon_outcome_months'] = dog_age(df_filtered['Age upon Outcome'])[0]
df_filtered['age_upon_outcome_years'] = dog_age(df_filtered['Age upon Outcome'])[1]

In [13]:
df_filtered.drop(columns = ['DateTimeIntake', 'DateTimeOutcome','Age upon Intake', 'Age upon Outcome'], inplace = True)
df_filtered.head(2)

,Animal ID,Intake Type,Intake Condition,Animal Type,Breed,Color,Outcome Type,Sex upon Outcome,days_in_shelter,age_upon_intake_months,age_upon_intake_years,age_upon_outcome_months,age_upon_outcome_years
0,A006100,Public Assist,Normal,Dog,Spinone Italiano Mix,Yellow/White,Return to Owner,Neutered Male,2.0,72.0,6.0,72.0,6.0
1,A047759,Owner Surrender,Normal,Dog,Dachshund,Tricolor,Transfer,Neutered Male,5.0,120.0,10.0,120.0,10.0


### Categorical features

In [14]:
# Intake Type (FERNANDO)
# Intake Condition (FERNANDO)
# Color (FERNANDO)
# Outcome Type (FERNANDO)
# Outcome Subtype (FERNANDO)
# Sex upon Outcome (GONÇALO)
# Breed (GONÇALO)

#### Sex upon Outcome   
Dividing the sex and the status of dog into 2 columns

In [15]:
df_filtered['Sex upon Outcome'].value_counts()

Neutered Male    36360
Spayed Female    33354
Intact Male      14439
Intact Female    14395
Unknown           4177
Name: Sex upon Outcome, dtype: int64

In [16]:
# Defining functions to see if the animal is neutered/spayed or not and if the animal is male or female

def neutered_animals(df):
    animal_neutered = []
    for animal in df:
        if 'Neutered' in animal or 'Spayed' in animal:
            animal_neutered.append(1)
        elif 'Unkown' in animal:
            animal_neutered.append(np.nan)
        else:
            animal_neutered.append(0)
    return animal_neutered

def male_animals(df):
    male_animal = []
    for animal in df:
        if 'Male' in animal:
            male_animal.append(1)
        elif 'Female' in animal:
            male_animal.append(0)
        else:
            male_animal.append(np.nan)
    return male_animal

In [17]:
df_filtered['neutered_or_spayed'] = neutered_animals(df_filtered['Sex upon Outcome'])
df_filtered['male_or_female'] = male_animals(df_filtered['Sex upon Outcome'])
df_filtered.drop(columns = ['Sex upon Outcome'], inplace = True)

#### Breed   
Merging all the breeds that are mixed

In [18]:
df_filtered['Breed'].value_counts()

Domestic Shorthair Mix                     29240
Domestic Shorthair                          6646
Pit Bull Mix                                6372
Labrador Retriever Mix                      5717
Chihuahua Shorthair Mix                     5556
                                           ...  
Bluetick Hound/Treeing Walker Coonhound        1
Lhasa Apso/Yorkshire Terrier                   1
Basset Hound/Golden Retriever                  1
Jack Russell Terrier/Chihuahua Longhair        1
Smooth Fox Terrier/French Bulldog              1
Name: Breed, Length: 2417, dtype: int64

In [19]:
print(f"The number of CAT BREEDS in df_filtered is {df_filtered[df_filtered['Animal Type'] == 'Cat']['Breed'].nunique()}")
print(f"The number of DOG BREEDS in df_filtered is {df_filtered[df_filtered['Animal Type'] == 'Dog']['Breed'].nunique()}")

The number of CAT BREEDS in df_filtered is 100
The number of DOG BREEDS in df_filtered is 2317


In [20]:
# Defining a function to reduce the number of breeds to mixed and pure

def breed(df):
    breeds = []
    for breed in df:
        if 'Mix' in breed or '/' in breed or 'Domestic' in breed:
            breeds.append('Mixed')
        else:
            breeds.append('Pure')
    return breeds

In [21]:
df_filtered['breed'] = breed(df_filtered['Breed'])
df_filtered.drop(columns = 'Breed', inplace = True)

In [22]:
df_filtered.head(2)

,Animal ID,Intake Type,Intake Condition,Animal Type,Color,Outcome Type,days_in_shelter,age_upon_intake_months,age_upon_intake_years,age_upon_outcome_months,age_upon_outcome_years,neutered_or_spayed,male_or_female,breed
0,A006100,Public Assist,Normal,Dog,Yellow/White,Return to Owner,2.0,72.0,6.0,72.0,6.0,1,1.0,Mixed
1,A047759,Owner Surrender,Normal,Dog,Tricolor,Transfer,5.0,120.0,10.0,120.0,10.0,1,1.0,Pure


#### Color

In [23]:
df_filtered.Color.value_counts()

Black/White                    10916
Black                           8910
Brown Tabby                     6680
White                           3629
Brown Tabby/White               3384
                               ...  
Black Tabby/Orange                 1
Blue Tabby/Orange                  1
Tricolor/Tricolor                  1
Brown Tabby/Orange Tabby           1
Brown Brindle/Black Brindle        1
Name: Color, Length: 562, dtype: int64

In [24]:
def colors(df):
    colors = []
    for color in df:
        if '/' in color:
            colors.append('Bicolor')
        elif 'Tricolor' in color or 'Calico' in color or 'Torbie' in color or 'Tortie' in color:
            colors.append('Tricolor')
        elif 'Agouti' in color or 'Black' in color or 'Blue' in color or 'Buff' in color or 'Brown' in color \
        or 'Chocolate' in color or 'Orange' in color or 'Gray' in color or 'Lilac' in color or 'Liver' in color \
        or 'Orange' in color or 'Red' in color or 'Sable' in color:
            colors.append('Dark')
        elif 'Apricot' in color or 'Cream' in color or 'Fawn' in color or 'Flame' in color or 'Gold' in color \
        or 'Lynx' in color or 'Pink' in color or 'Seal' in color or 'Silver' in color or 'Tan' in color \
        or 'White' in color or 'Yellow' in color:
            colors.append('Light')
    return colors

In [25]:
df_filtered['color'] = colors(df_filtered['Color'])
df_filtered.drop(columns = 'Color')

,Animal ID,Intake Type,Intake Condition,Animal Type,Outcome Type,days_in_shelter,age_upon_intake_months,age_upon_intake_years,age_upon_outcome_months,age_upon_outcome_years,neutered_or_spayed,male_or_female,breed,color
0,A006100,Public Assist,Normal,Dog,Return to Owner,2.0,72.0,6.0,72.0,6.0,1,1.0,Mixed,Bicolor
1,A047759,Owner Surrender,Normal,Dog,Transfer,5.0,120.0,10.0,120.0,10.0,1,1.0,Pure,Tricolor
2,A134067,Public Assist,Injured,Dog,Return to Owner,1.0,192.0,16.0,192.0,16.0,1,1.0,Pure,Bicolor
3,A141142,Stray,Aged,Dog,Return to Owner,1.0,180.0,15.0,180.0,15.0,1,0.0,Mixed,Bicolor
4,A163459,Stray,Normal,Dog,Return to Owner,1.0,180.0,15.0,180.0,15.0,0,0.0,Mixed,Bicolor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110852,A829991,Stray,Normal,Dog,Return to Owner,2.0,24.0,2.0,24.0,2.0,0,1.0,Mixed,Bicolor
110853,A829992,Stray,Normal,Dog,Return to Owner,2.0,24.0,2.0,24.0,2.0,1,0.0,Mixed,Bicolor
110861,A830034,Stray,Normal,Dog,Return to Owner,1.0,24.0,2.0,24.0,2.0,1,1.0,Mixed,Bicolor
110874,A830080,Owner Surrender,Normal,Dog,Rto-Adopt,2.0,12.0,1.0,12.0,1.0,1,1.0,Pure,Bicolor


In [436]:
import pandas as pd
import numpy as np
import re


def get_data():
    # Read CSV
    df_intakes = pd.read_csv('../raw_data/Austin_Animal_Center_Intakes.csv', parse_dates=['DateTime'])
    df_outcomes = pd.read_csv('../raw_data/Austin_Animal_Center_Outcomes.csv', parse_dates=['DateTime'])
    df_straymap = pd.read_csv('../raw_data/Austin_Animal_Center_Stray_Map.csv')

    # Drop Duplicates
    df_intakes.drop_duplicates(inplace = True)
    df_outcomes.drop_duplicates(inplace = True)

    #Dropping Irrelevant Features
    df_intakes.drop(columns = ['Name','MonthYear','Found Location'], inplace = True)
    df_intakes.rename(columns = {'DateTime':'DateTimeIntake'}, inplace = True)
    df_outcomes.drop(columns = ['Name','MonthYear','Date of Birth','Breed','Color','Animal Type', 'Outcome Subtype'], inplace = True)
    df_outcomes.rename(columns = {'DateTime':'DateTimeOutcome'}, inplace = True)

    # Sorting all the intakes and outcomes based on Animal ID and DateTime
    df_intakes.sort_values(by = ['Animal ID', 'DateTimeIntake'], ascending = [True, True], inplace = True)
    df_outcomes.sort_values(by = ['Animal ID','DateTimeOutcome'], ascending = [True,True], inplace = True) 

    # Dropping all the animals that were more than one time in the shelter
    df_intakes.drop_duplicates(subset = 'Animal ID', inplace = True)
    df_outcomes.drop_duplicates(subset = 'Animal ID', inplace = True)
    
    # Merging the datasets
    df_merged = pd.merge(left = df_intakes, right = df_outcomes, how = 'left', on = ['Animal ID'])

    # Filtering only dogs
    df_filtered = df_merged[(df_merged['Animal Type'] == 'Dog') | (df_merged['Animal Type'] == 'Cat')].copy()

    # Calculating the number of days a dog stays in shelter
    df_filtered['days_in_shelter'] = np.ceil((df_filtered['DateTimeOutcome'] - df_filtered['DateTimeIntake']) / np.timedelta64(24,'h'))

    # Dropping all the the negatives values (errors in merging datasets) and null values (dogs that are still in shelter)
    df_filtered = df_filtered[df_filtered.days_in_shelter > 0]

    #Dropping rows with NaNs
    df_filtered.dropna(inplace = True)

    # Transforming ages in string to integer in months
    def dog_age(df):
        dog_months = []
        dog_years = []
        for age in df:
            if 'years' in age or 'year' in age:
                years = int(re.findall('(-?\d+)',age)[0])
                if years < 0:
                    dog_months.append(np.nan)
                    dog_years.append(np.nan)
                else:
                    dog_months.append(years*12)
                    dog_years.append(years)
            elif 'months' in age or 'month' in age:
                months = int(re.findall('(-?\d+)',age)[0])
                years = int(re.findall('(-?\d+)',age)[0])
                if months < 0:
                    dog_months.append(np.nan)
                else:
                    dog_months.append(months)
                    dog_years.append(1)
            else:
                weeks_or_days = int(re.findall('(-?\d+)',age)[0])
                if weeks_or_days < 0:
                    dog_months.append(np.nan)
                else:
                    dog_months.append(1)
                    dog_years.append(1)
        return [dog_months, dog_years]

    df_filtered['age_upon_intake_months'] = dog_age(df_filtered['Age upon Intake'])[0]
    df_filtered['age_upon_intake_years'] = dog_age(df_filtered['Age upon Intake'])[1]
    df_filtered['age_upon_outcome_months'] = dog_age(df_filtered['Age upon Outcome'])[0]
    df_filtered['age_upon_outcome_years'] = dog_age(df_filtered['Age upon Outcome'])[1]
    df_filtered.drop(columns = ['DateTimeIntake', 'DateTimeOutcome','Age upon Intake', 'Age upon Outcome'], inplace = True)

    # Defining functions to see if the animal is neutered/spayed
    def neutered_animals(df):
        animal_neutered = []
        for animal in df:
            if 'Neutered' in animal or 'Spayed' in animal:
                animal_neutered.append(1)
            elif 'Unkown' in animal:
                animal_neutered.append(np.nan)
            else:
                animal_neutered.append(0)
        return animal_neutered

    # or not and if the animal is male or female
    def male_animals(df):
        male_animal = []
        for animal in df:
            if 'Male' in animal:
                male_animal.append(1)
            elif 'Female' in animal:
                male_animal.append(0)
            else:
                male_animal.append(np.nan)
        return male_animal

    df_filtered['neutered_or_spayed_outcome'] = neutered_animals(df_filtered['Sex upon Outcome'])
    df_filtered['male_or_female_outcome'] = male_animals(df_filtered['Sex upon Outcome'])
    df_filtered['neutered_or_spayed_intake'] = neutered_animals(df_filtered['Sex upon Intake'])
    df_filtered['male_or_female_intake'] = male_animals(df_filtered['Sex upon Intake'])
    df_filtered.drop(columns = ['Sex upon Outcome', 'Sex upon Intake'], inplace = True)

    # Defining a function to reduce the number of breeds to mixed and pure
    def breed(df):
        breeds = []
        for breed in df:
            if 'Mix' in breed or '/' in breed or 'Domestic' in breed:
                breeds.append('Mixed')
            else:
                breeds.append('Pure')
        return breeds

    df_filtered['Breed'] = breed(df_filtered['Breed'])

    # Defining a function to reduce the number of colors
    def colors(df):
    colors = []
    for color in df:
        if '/' in color:
            colors.append('Bicolor')
        elif 'Tricolor' in color or 'Calico' in color or 'Torbie' in color or 'Tortie' in color:
            colors.append('Tricolor')
        elif 'Agouti' in color or 'Black' in color or 'Blue' in color or 'Buff' in color or 'Brown' in color \
        or 'Chocolate' in color or 'Orange' in color or 'Gray' in color or 'Lilac' in color or 'Liver' in color \
        or 'Orange' in color or 'Red' in color or 'Sable' in color:
            colors.append('Dark')
        elif 'Apricot' in color or 'Cream' in color or 'Fawn' in color or 'Flame' in color or 'Gold' in color \
        or 'Lynx' in color or 'Pink' in color or 'Seal' in color or 'Silver' in color or 'Tan' in color \
        or 'White' in color or 'Yellow' in color:
            colors.append('Light')
    return colors
  
    df_filtered['color'] = colors(df_filtered['Color'])
    df_filtered.drop(columns = 'Color', inplace = True)
    return df_filtered